In [ ]:
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!pip install mahotas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
find: ‘/proc/59/task/59/net’: Invalid argument
find: ‘/proc/59/net’: Invalid argument
/usr/local/lib/python3.8/dist-packages/jaxlib/cuda/nvvm/libdevice
/usr/local/cuda-11.8/nvvm/libdevice
find: ‘/proc/59/task/59/net’: Invalid argument
find: ‘/proc/59/net’: Invalid argument
/usr/local/cuda-11.8/nvvm/lib64/libnvvm.so
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 80.5 MB/s eta 0:00:00


In [ ]:
#Loading part
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
downloaded = drive.CreateFile({'id': '1WjbWaN4aZNtdvVmxG_D4OiBR2hLeadh4'})   # replace the id with id of file you want to access
downloaded.GetContentFile('Detectedface.zip')        # replace the file name with your file
!unzip Detectedface

Streaming output truncated to the last 5000 lines.
  inflating: ImposterFace/0011/0011_01_03_03_82.jpg  
  inflating: ImposterFace/0011/0011_01_08_03_136.jpg  
  inflating: ImposterFace/0011/0011_01_04_03_171.jpg  
  inflating: ImposterFace/0011/0011_01_03_03_300.jpg  
  inflating: ImposterFace/0011/0011_01_08_03_453.jpg  
  inflating: ImposterFace/0011/0011_01_08_03_447.jpg  
  inflating: ImposterFace/0011/0011_01_01_03_217.jpg  
  inflating: ImposterFace/0011/0011_01_03_03_103.jpg  
  inflating: ImposterFace/0011/0011_01_04_03_372.jpg  
  inflating: ImposterFace/0011/0011_01_04_03_24.jpg  
  inflating: ImposterFace/0011/0011_01_02_03_386.jpg  
  inflating: ImposterFace/0011/0011_01_01_03_149.jpg  
  inflating: ImposterFace/0011/0011_01_08_03_44.jpg  
  inflating: ImposterFace/0011/0011_01_03_03_275.jpg  
  inflating: ImposterFace/0011/0011_01_02_03_184.jpg  
  inflating: ImposterFace/0011/0011_01_01_03_411.jpg  
  inflating: ImposterFace/0011/0011_01_01_03_375.jpg  
  inflating: Impo

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from scipy import signal
import scipy.io
import math
import sys
import time
from IPython.display import HTML, display
from IPython.display import clear_output
import PIL
from PIL import Image
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8
# import mahotas
# import mahotas.demos
from pylab import gray, imshow, show
import numpy as np

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)            
    return allFiles

In [ ]:
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as tf
import skimage
import skimage.color as skcolor
import skimage.io as skio
from PIL import Image
import cv2
import numpy as np
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
import matplotlib.pylab as plt

In [ ]:
def show(im):
    plt.imshow(im)
    plt.show()
    

In [ ]:
def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from numba import int32, float32    # import the types
from numba.experimental import jitclass


class Human:

    
    def __init__(self):
        self.img = 0
    
    
    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    
    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        # image = rgb2lab(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        height, width, xx = image.shape

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
        b, g, r = cv2.split(image)

        # 3 blank rgb
        img_lbpr = np.zeros((height, width), np.uint8)
        img_lbpg = np.zeros((height, width), np.uint8)
        img_lbpb = np.zeros((height, width), np.uint8)
        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
                img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
                img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # into a coloured/multi-channeled image
        image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        path = '/content/HSV_LBP/Imposter_Face'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/HSV_LBP/Imposter_Face'
        os.chdir(directory)
        cv2.imwrite(strr[25:], image_merge)
#         cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human = Human()
# split
# human.rgblbp('/content/ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg')

In [ ]:
path_list = getListOfFiles("/content/ClientFace")

# len(path_list)
for i in path_list:
  human.rgblbp(i)

In [ ]:
!zip -r /content/Imposter_Face.zip /content/HSV_LBP/Imposter_Face
from google.colab import files
files.download("/content/Imposter_Face.zip")

Streaming output truncated to the last 5000 lines.
updating: content/HSV_LBP/Imposter_Face/0009_01_01_03_81.jpg (deflated 0%)
updating: content/HSV_LBP/Imposter_Face/0012_01_04_03_393.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0004_01_04_03_180.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0009_01_01_02_495.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0004_01_03_03_462.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0005_00_00_01_251.jpg (deflated 0%)
updating: content/HSV_LBP/Imposter_Face/0002_01_01_03_271.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0012_01_01_03_461.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0008_00_00_01_196.jpg (deflated 0%)
updating: content/HSV_LBP/Imposter_Face/0014_01_04_03_176.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0009_01_03_03_342.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Face/0006_00_04_03_389.jpg (deflated 1%)
updating: content/HSV_LBP/Imposter_Fac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Python program to explain Merging of Channels

# Importing cv2
import cv2
import numpy as np
from matplotlib import pyplot as plt


class Human2:

    def __init__(self):
        self.img = 0

    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human2.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human2.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human2.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human2.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human2.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human2.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human2.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human2.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        # image = rgb2lab(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        height, width, xx = image.shape

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
        b, g, r = cv2.split(image)

        # 3 blank rgb
        img_lbpr = np.zeros((height, width), np.uint8)
        img_lbpg = np.zeros((height, width), np.uint8)
        img_lbpb = np.zeros((height, width), np.uint8)
        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbpr[i, j] = Human2.lbp_calculated_pixel8(r, i, j)
                img_lbpg[i, j] = Human2.lbp_calculated_pixel8(g, i, j)
                img_lbpb[i, j] = Human2.lbp_calculated_pixel8(b, i, j)

        # into a coloured/multi-channeled image
        image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # path = '/content/LBP_outputs/LBP_RGB/dev'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/HSV_LBP/test'
        os.chdir(directory)
        cv2.imwrite(strr[35:], image_merge)
        cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human2 = Human2()
# split
# human.rgblbp('/content/ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg')

In [ ]:
path_list = getListOfFiles("/content/ExtractedRGBOulu_NPU/test")

for i in path_list:
  human2.rgblbp(i)

In [ ]:
!zip -r /content/test.zip /content/HSV_LBP/test
from google.colab import files
files.download("/content/test.zip")

In [ ]:
# Python program to explain Merging of Channels

# Importing cv2
import cv2
import numpy as np
from matplotlib import pyplot as plt


class Human3:

    def __init__(self):
        self.img = 0

    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human3.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human3.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human3.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human3.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human3.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human3.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human3.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human3.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        image = rgb2lab(image)
        height, width, xx = image.shape

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
        b, g, r = cv2.split(image)

        # 3 blank rgb
        img_lbpr = np.zeros((height, width), np.uint8)
        img_lbpg = np.zeros((height, width), np.uint8)
        img_lbpb = np.zeros((height, width), np.uint8)
        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbpr[i, j] = Human3.lbp_calculated_pixel8(r, i, j)
                img_lbpg[i, j] = Human3.lbp_calculated_pixel8(g, i, j)
                img_lbpb[i, j] = Human3.lbp_calculated_pixel8(b, i, j)

        # into a coloured/multi-channeled image
        image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # path = '/content/LBP_outputs/LBP_RGB/dev'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/HSV_LBP/train'
        os.chdir(directory)
        cv2.imwrite(strr[36:], image_merge)
        cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human3 = Human3()
# split
# human.rgblbp('/content/ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg')

In [ ]:
path_list = getListOfFiles("/content/ExtractedRGBOulu_NPU/train")

for i in path_list:
  human3.rgblbp(i)

In [ ]:
!zip -r /content/train.zip /content/HSV_LBP/train
from google.colab import files
files.download("/content/train.zip")